In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler ## By using this line we can see how langchain works

In [2]:
MODEL = "gpt-4o-mini"
db_name = "HII_vector_db"

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


### Read in documents using LangChain's loaders

In [8]:
## We can use this code to read markdwon files
folders = glob.glob("HII_Data/*")
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [10]:
len(chunks)

8

In [11]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: Student_Life, About, ProgrammsandCourses, Profiles, Careers


In [12]:
embeddings = OpenAIEmbeddings()

In [13]:
## Delete if already exists
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

### Create the VectorDB with Chroma

In [14]:
vectorstore = Chroma.from_documents(documents = chunks, 
                                    embedding = embeddings, 
                                    persist_directory = db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 8 documents


In [15]:
## Get one vector and find how many dimensions it has
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


### Chat Application (RAG pipeline with LangChain)

In [20]:
## create a new Chat with OpenAI (model)
llm = ChatOpenAI(temperature = 0.7, 
                 model_name = MODEL)

## set up the conversation memory for the chat (Memory)
memory = ConversationBufferMemory(
    memory_key = 'chat_history', 
    return_messages = True)

## the retriever is an abstraction over the VectorStore that will be used during RAG (Retreiver)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm = llm, 
    retriever = retriever, 
    memory = memory,
    callbacks = [StdOutCallbackHandler()]) ## By using this line, we can see how langchain works

In [17]:
query = "What is HII"
result = conversation_chain.invoke({"question":query})
print(result["answer"])



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
# About Horizon International Institute

Horizon International Institute (HII) is a globally recognized center for higher education established in 1995.  
Our mission is to provide transformative education through innovation, research, and strong industry partnerships.

## Vision
To empower students with knowledge, creativity, and leadership skills that shape the future.

## Mission
- Deliver high-quality, accessible education.  
- Foster research and innovation.  
- Build a community that values diversity, ethics, and collaboration.

## Accreditations
- Accredited by the National Board of Higher Education (NBHE).  
- International partnerships wi

In [21]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [22]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
# About Horizon International Institute

Horizon International Institute (HII) is a globally recognized center for higher education established in 1995.  
Our mission is to provide transformative education through innovation, research, and strong industry partnerships.

## Vision
To empower students with knowledge, creativity, and leadership skills that shape the future.

## Mission
- Deliver high-quality, accessible education.  
- Foster research and innovation.  
- Build a community that values diversity, ethics, and collaboration.

## Accreditations
- Accredited by the National Board of Higher Education (NBHE).  
- International partnerships wi

### If the RAG pipeline does not give the correct answers, we can do followings

* Change the chunk size and look in their overlapping
* Instead of giving chunks to the context, pass the full document to it
* Set the number of chunks in the RAG Pipeline as follows (Recommended)

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

#### the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)